In [1]:
import os

In [2]:
%pwd

'd:\\DS\\Projects\\DL\\Cars-Tanks-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\DS\\Projects\\DL\\Cars-Tanks-Classification'

In [5]:
# You have to provide yours not mine!

#os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Abdallah-Ahmed-98/Cars-Tanks-Classification.mlflow"
#os.environ["MLFLOW_TRACKING_USERNAME"]="Abdallah-Ahmed-98"
#os.environ["MLFLOW_TRACKING_PASSWORD"]="****************************************" 

In [6]:
import tensorflow as tf

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    root_dir: Path
    metric_file_name: Path
    model_path: Path
    dataset_dir: Path
    confusion_matrix_file_path: Path
    confusion_matrix_data_file_path: Path
    roc_curve_file_path: Path
    roc_data_file_path: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int
    params_registered_model_name: str

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [9]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:

        evaluation = self.config.evaluation
        params = self.params
        
        create_directories([
            Path(evaluation.root_dir)
        ])

        eval_config = EvaluationConfig(
            root_dir=evaluation.root_dir,
            model_path=evaluation.model_path,
            dataset_dir=evaluation.dataset_dir,
            metric_file_name = evaluation.metric_file_name,
            confusion_matrix_file_path = evaluation.confusion_matrix_file_path,
            confusion_matrix_data_file_path = evaluation.confusion_matrix_data_file_path,
            roc_curve_file_path = evaluation.roc_curve_file_path,
            roc_data_file_path= evaluation.roc_data_file_path,
            mlflow_uri="",
            all_params=params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE,
            params_registered_model_name=self.params.REGISTERED_MODEL_NAME
        )
        return eval_config
    




In [10]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import label_binarize

In [11]:

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1.0 / 255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bicubic"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.dataset_dir,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        # Load the model
        self.model = self.load_model(self.config.model_path)

        # Prepare the validation generator
        self._valid_generator()

        # Evaluate on the validation set
        self.score = self.model.evaluate(self.valid_generator, verbose=1)

        # Generate predictions
        y_pred = self.model.predict(self.valid_generator)
        y_pred_classes = np.argmax(y_pred, axis=1)
        y_true = self.valid_generator.classes

        # Calculate precision and recall
        self.report = classification_report(y_true, y_pred_classes, output_dict=True)
        precision = self.report["macro avg"]["precision"]
        recall = self.report["macro avg"]["recall"]

        # Save scores
        self.save_score()

        # Save and log confusion matrix
        self.save_confusion_matrix(y_true, y_pred_classes)

        # Save and log ROC curve
        self.save_roc_curve(y_true, y_pred)

        return precision, recall

    def save_score(self):
        scores = {
            "loss": self.score[0],
            "accuracy": self.score[1],
            "precision": self.report["macro avg"]["precision"],
            "recall": self.report["macro avg"]["recall"]
        }
        save_json(path=Path(self.config.metric_file_name), data=scores)

    def save_confusion_matrix(self, y_true, y_pred_classes):
        """
        Save the normalized confusion matrix as a JSON file and as a heatmap image.
    
        Args:
            y_true (np.ndarray): True labels.
            y_pred_classes (np.ndarray): Predicted class labels.
        """
        # Compute the confusion matrix
        cm = confusion_matrix(y_true, y_pred_classes)

        # Normalize the confusion matrix
        cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm_normalized = np.nan_to_num(cm_normalized, nan=0.0)  # Handle cases where rows sum to 0

        # Save normalized confusion matrix as JSON
        cm_data_file = Path(self.config.confusion_matrix_data_file_path)
        cm_data = {
            "confusion_matrix": cm.tolist(),
            "normalized_confusion_matrix": cm_normalized.tolist(),
        }
        cm_data_file.parent.mkdir(parents=True, exist_ok=True)  # Ensure the directory exists
        save_json(cm_data_file, cm_data)

        # Save the normalized confusion matrix as a heatmap
        plt.figure(figsize=(10, 7))
        sns.heatmap(cm_normalized, annot=True, fmt=".2f", cmap="Blues", cbar=True)
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title("Normalized Confusion Matrix")
        cm_image_file = Path(self.config.confusion_matrix_file_path)
        plt.savefig(cm_image_file, bbox_inches='tight', dpi=300)
        plt.close()

        return cm_data_file, cm_image_file



    def save_roc_curve(self, y_true, y_pred):
        # Binarize the labels for ROC curve calculation
        y_true_binarized = label_binarize(y_true, classes=np.unique(y_true))
        n_classes = y_true_binarized.shape[1]

        # Calculate ROC curve and AUC for each class
        fpr = {}
        tpr = {}
        roc_auc = {}

        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(y_true_binarized[:, i], y_pred[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])

        # Save ROC data for all classes
        roc_data_file = Path(self.config.roc_data_file_path)
        roc_data = {
            f"class_{i}": {"fpr": fpr[i].tolist(), "tpr": tpr[i].tolist(), "auc": roc_auc[i]}
            for i in range(n_classes)
        }
        save_json(roc_data_file, roc_data)

        # Plot ROC curve for each class
        plt.figure()
        for i in range(n_classes):
            plt.plot(
                fpr[i], tpr[i], lw=2, label=f"Class {i} (AUC = {roc_auc[i]:.2f})"
            )
        plt.plot([0, 1], [0, 1], color="gray", lw=2, linestyle="--")
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.title("Receiver Operating Characteristic")
        plt.legend(loc="lower right")
        roc_image_file=Path(self.config.roc_curve_file_path)
        plt.savefig(roc_image_file)
        plt.close()

        return roc_data_file, roc_image_file


    def log_into_mlflow(self, precision, recall):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            # Log parameters and metrics
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {
                    "loss": self.score[0],
                    "accuracy": self.score[1],
                    "precision": precision,
                    "recall": recall,
                }
            )

            # Log confusion matrix
            cm_file, cm_image_file = self.save_confusion_matrix(
                self.valid_generator.classes, np.argmax(self.model.predict(self.valid_generator), axis=1)
            )
            mlflow.log_artifact(str(cm_file))
            mlflow.log_artifact(cm_image_file)

            # Log ROC curve
            roc_file, roc_image_file = self.save_roc_curve(
                self.valid_generator.classes, self.model.predict(self.valid_generator)
            )
            mlflow.log_artifact(str(roc_file))
            mlflow.log_artifact(roc_image_file)

            # Register the model
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name=self.config.params_registered_model_name)
            else:
                mlflow.keras.log_model(self.model, "model")



In [12]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    precision, recall = evaluation.evaluation()
    evaluation.log_into_mlflow(precision, recall)

except Exception as e:
   raise e

[2025-01-05 22:58:08,215: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-05 22:58:08,237: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-05 22:58:08,239: INFO: common: created directory at: artifacts]
[2025-01-05 22:58:08,240: INFO: common: created directory at: artifacts\evaluation]
Found 450 images belonging to 2 classes.
29/29 [==============================] - 37s 1s/step
[2025-01-05 22:59:20,748: INFO: common: json file saved at: artifacts\evaluation\metrics.json]
[2025-01-05 22:59:20,760: INFO: common: json file saved at: artifacts\evaluation\confusion_matrix.json]
[2025-01-05 22:59:21,549: INFO: common: json file saved at: artifacts\evaluation\roc_data.json]


2025/01/05 22:59:21 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



29/29 [==============================] - 33s 1s/step
[2025-01-05 22:59:55,841: INFO: common: json file saved at: artifacts\evaluation\confusion_matrix.json]
29/29 [==============================] - 34s 1s/step
[2025-01-05 23:00:32,372: INFO: common: json file saved at: artifacts\evaluation\roc_data.json]


2025/01/05 23:00:33 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-01-05 23:00:54,062: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 53). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\abdot\AppData\Local\Temp\tmp0ebems86\model\data\model\assets
[2025-01-05 23:01:00,867: INFO: builder_impl: Assets written to: C:\Users\abdot\AppData\Local\Temp\tmp0ebems86\model\data\model\assets]


2025/01/05 23:01:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ResNet50V2Model'.
2025/01/05 23:02:47 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ResNet50V2Model, version 1
Created version '1' of model 'ResNet50V2Model'.


🏃 View run delightful-eel-257 at: https://dagshub.com/Abdallah-Ahmed-98/Cars-Tanks-Classification.mlflow/#/experiments/0/runs/50606d2b13ae4c15bbe885c74f30513d
🧪 View experiment at: https://dagshub.com/Abdallah-Ahmed-98/Cars-Tanks-Classification.mlflow/#/experiments/0
